<a href="https://colab.research.google.com/github/Jazzystic/data_code_migration_GFG/blob/main/migraci%C3%B3n_hist%C3%B3rico_consolidado_GFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Subir archivo
print("Selecciona el archivo data_consolidado.csv:")
uploaded = files.upload()

Selecciona el archivo data_consolidado.csv:


Saving data_consolidado.csv to data_consolidado.csv


In [2]:
import pandas as pd

# Leer el CSV
df = pd.read_csv('data_consolidado.csv',
                 encoding='utf-8',
                 on_bad_lines='warn',
                 engine='python')

print(f"Total de filas: {len(df)}")
print(f"\nColumnas detectadas ({len(df.columns)}):")
print(df.columns.tolist())
print("\nPrimeras 3 filas:")
print(df.head(3))
print("\nValores nulos por columna:")
print(df.isnull().sum())
print("\nTipos de datos:")
print(df.dtypes)

Total de filas: 20638

Columnas detectadas (58):
['Week', 'WeekDate', 'GFG', 'Audit for:', 'CS', 'Series', 'CS Hotels Names', 'Rank All', 'Total', 'Market', 'TA.Reviews>', 'TA.Excellent', 'TA.Very Good', 'TA.Average', 'TA.Poor', 'TA.Terrible', 'TA.Ratings>', 'TA.Location', 'TA.Cleanliness', 'TA.Service', 'TA.Value', 'GG.Reviews>', 'GG.Excellent', 'GG.Very Good', 'GG.Average', 'GG.Poor', 'GG.Terrible', 'GG.Ratings>', 'GG.Location', 'GG.Cleanliness', 'GG.Service', 'GG.Value', 'EX.Reviews>', 'EX.Excellent', 'EX.Very Good', 'EX.Average', 'EX.Poor', 'EX.Terrible', 'EX.Ratings>', 'EX.Location', 'EX.Cleanliness', 'EX.Service', 'EX.Value', 'BK.Reviews>', 'BK.Excellent', 'BK.Very Good', 'BK.Average', 'BK.Poor', 'BK.Terrible', 'BK.Ratings>', 'BK.Location', 'BK.Cleanliness', 'BK.Service', 'BK.Value', 'Instagram', 'Facebook', 'Twitter', 'Unnamed: 57']

Primeras 3 filas:
     Week    WeekDate  GFG                           Audit for:      CS  \
0  220801  01/08/2022  Yes  Dreams Bahia Mita Surf & S

In [3]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

# Limpiar datos
df_clean = df.copy()

# Eliminar columna vacía
df_clean = df_clean.drop(columns=['Unnamed: 57'], errors='ignore')

# Renombrar columnas para que coincidan con PostgreSQL
column_mapping = {
    'Week': 'week',
    'WeekDate': 'week_date',
    'GFG': 'active_selection',
    'CS': 'cs',
    'Series': 'series',
    'CS Hotels Names': 'cs_hotels_names',
    'Rank All': 'rank_all',
    'Total': 'total',
    'Market': 'market',
    'TA.Reviews>': 'ta_reviews',
    'TA.Excellent': 'ta_excellent',
    'TA.Very Good': 'ta_very_good',
    'TA.Average': 'ta_average',
    'TA.Poor': 'ta_poor',
    'TA.Terrible': 'ta_terrible',
    'TA.Ratings>': 'ta_ratings',
    'TA.Location': 'ta_location',
    'TA.Cleanliness': 'ta_cleanliness',
    'TA.Service': 'ta_service',
    'TA.Value': 'ta_value',
    'GG.Reviews>': 'gg_reviews',
    'GG.Excellent': 'gg_excellent',
    'GG.Very Good': 'gg_very_good',
    'GG.Average': 'gg_average',
    'GG.Poor': 'gg_poor',
    'GG.Terrible': 'gg_terrible',
    'GG.Ratings>': 'gg_ratings',
    'GG.Location': 'gg_location',
    'GG.Cleanliness': 'gg_cleanliness',
    'GG.Service': 'gg_service',
    'GG.Value': 'gg_value',
    'EX.Reviews>': 'ex_reviews',
    'EX.Excellent': 'ex_excellent',
    'EX.Very Good': 'ex_very_good',
    'EX.Average': 'ex_average',
    'EX.Poor': 'ex_poor',
    'EX.Terrible': 'ex_terrible',
    'EX.Ratings>': 'ex_ratings',
    'EX.Location': 'ex_location',
    'EX.Cleanliness': 'ex_cleanliness',
    'EX.Service': 'ex_service',
    'EX.Value': 'ex_value',
    'BK.Reviews>': 'bk_reviews',
    'BK.Excellent': 'bk_excellent',
    'BK.Very Good': 'bk_very_good',
    'BK.Average': 'bk_average',
    'BK.Poor': 'bk_poor',
    'BK.Terrible': 'bk_terrible',
    'BK.Ratings>': 'bk_ratings',
    'BK.Location': 'bk_location',
    'BK.Cleanliness': 'bk_cleanliness',
    'BK.Service': 'bk_service',
    'BK.Value': 'bk_value',
    'Instagram': 'instagram',
    'Facebook': 'facebook',
    'Twitter': 'twitter'
}

df_clean = df_clean.rename(columns=column_mapping)

# Convertir week_date a formato DATE
df_clean['week_date'] = pd.to_datetime(df_clean['week_date'], format='%d/%m/%Y', errors='coerce')
df_clean['week_date'] = df_clean['week_date'].apply(lambda x: x.date() if pd.notna(x) else None)

# Convertir columnas numéricas que DEBEN ser enteros
integer_cols = ['week', 'series', 'rank_all', 'total',
                'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                'instagram', 'facebook', 'twitter']

for col in integer_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].round(0).astype('Int64')

# Convertir columnas DECIMAL
decimal_cols = ['ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value']

for col in decimal_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Reemplazar <NA> con None para PostgreSQL
df_clean = df_clean.where(pd.notna(df_clean), None)

print(f"Datos preparados: {len(df_clean)} filas")
print("\nMuestra de datos limpios:")
print(df_clean.head(2))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos
columns_order = ['week', 'week_date', 'active_selection', 'cs', 'series', 'cs_hotels_names',
                 'rank_all', 'total', 'market',
                 'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                 'ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                 'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                 'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                 'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                 'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                 'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                 'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value',
                 'instagram', 'facebook', 'twitter']

data = [tuple(row) for row in df_clean[columns_order].to_numpy()]

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.data_consolidado_historico
(week, week_date, active_selection, cs, series, cs_hotels_names,
 rank_all, total, market,
 ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible,
 ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value,
 gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible,
 gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value,
 ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible,
 ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value,
 bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible,
 bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value,
 instagram, facebook, twitter)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("\nInsertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n{len(data)} filas insertadas exitosamente")

cursor.close()
conn.close()

print("\nMigracion completada!")

TypeError: cannot safely cast non-equivalent float64 to int64

In [8]:
# Verificar duplicados en el CSV
duplicados = df_clean.duplicated(subset=['week', 'week_date', 'cs_hotels_names'], keep=False)
print(f"Número de filas duplicadas: {duplicados.sum()}")

if duplicados.sum() > 0:
    print("\nPrimeras 10 filas duplicadas:")
    print(df_clean[duplicados][['week', 'week_date', 'cs_hotels_names', 'active_selection']].head(10))

    # Eliminar duplicados (mantener la primera ocurrencia)
    print(f"\nEliminando {duplicados.sum()} duplicados...")
    df_clean = df_clean.drop_duplicates(subset=['week', 'week_date', 'cs_hotels_names'], keep='first')
    print(f"Filas después de eliminar duplicados: {len(df_clean)}")
else:
    print("No hay duplicados en el CSV")

Número de filas duplicadas: 8466

Primeras 10 filas duplicadas:
     week   week_date                             cs_hotels_names  \
0  220801  2022-08-01   Dreams Bahia Mita Surf & Spa Resort (363)   
1  220801  2022-08-01  Secrets Bahia Mita Surf & Spa Resort (278)   
2  220801  2022-08-01       The St. Regis Punta Mita Resort (120)   
3  220801  2022-08-01        Four Seasons Resort Punta Mita (205)   
4  220801  2022-08-01                       W Punta de Mita (119)   
5  220801  2022-08-01                  Conrad Punta de Mita (324)   
6  220801  2022-08-01                                Casa De Mita   
7  220801  2022-08-01                     Hotel La Quinta del Sol   
8  220801  2022-08-01              Four Seasons Resort Punta Mita   
9  220801  2022-08-01       Marival Armony Luxury Resort & Suites   

  active_selection  
0              Yes  
1               No  
2               No  
3               No  
4               No  
5               No  
6               No  
7       

In [9]:
# Recargar el CSV original para ver los duplicados completos
df_original = pd.read_csv('data_consolidado.csv', encoding='utf-8', on_bad_lines='warn', engine='python')

# Aplicar las mismas transformaciones de nombres
df_original = df_original.rename(columns=column_mapping)
df_original = df_original.drop(columns=['Unnamed: 57'], errors='ignore')

# Encontrar duplicados
duplicados = df_original.duplicated(subset=['week', 'week_date', 'cs_hotels_names'], keep=False)

print(f"Total de filas duplicadas: {duplicados.sum()}")
print("\nPrimeras 20 filas duplicadas (TODAS las columnas):")
print(df_original[duplicados].head(20))

# Ver si TODAS las columnas son idénticas o solo esas 3
print("\n¿Son filas COMPLETAMENTE idénticas?")
duplicados_completos = df_original.duplicated(keep=False)
print(f"Filas completamente idénticas: {duplicados_completos.sum()}")

Total de filas duplicadas: 8466

Primeras 20 filas duplicadas (TODAS las columnas):
      week   week_date active_selection                           Audit for:  \
0   220801  01/08/2022              Yes  Dreams Bahia Mita Surf & Spa Resort   
1   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
2   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
3   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
4   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
5   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
6   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
7   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
8   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
9   220801  01/08/2022               No  Dreams Bahia Mita Surf & Spa Resort   
10  220801  01/08/2022              

In [ ]:
# Preparar datos
columns_order = ['week', 'week_date', 'active_selection', 'cs', 'series', 'cs_hotels_names',
                 'rank_all', 'total', 'market',
                 'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                 'ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                 'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                 'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                 'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                 'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                 'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                 'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value',
                 'instagram', 'facebook', 'twitter']

# Convertir DataFrame a lista de tuplas
data = []
for _, row in df_clean[columns_order].iterrows():
    tuple_data = []
    for val in row:
        if pd.isna(val) or val is pd.NA:
            tuple_data.append(None)
        else:
            tuple_data.append(val)
    data.append(tuple(tuple_data))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.data_consolidado_historico
(week, week_date, active_selection, cs, series, cs_hotels_names,
 rank_all, total, market,
 ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible,
 ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value,
 gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible,
 gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value,
 ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible,
 ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value,
 bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible,
 bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value,
 instagram, facebook, twitter)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("Insertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"{len(data)} filas insertadas exitosamente")

cursor.close()
conn.close()

print("Migracion completada!")

In [7]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

# Limpiar datos
df_clean = df.copy()

# Eliminar columna vacía
df_clean = df_clean.drop(columns=['Unnamed: 57'], errors='ignore')

# Renombrar columnas para que coincidan con PostgreSQL
column_mapping = {
    'Week': 'week',
    'WeekDate': 'week_date',
    'GFG': 'active_selection',
    'CS': 'cs',
    'Series': 'series',
    'CS Hotels Names': 'cs_hotels_names',
    'Rank All': 'rank_all',
    'Total': 'total',
    'Market': 'market',
    'TA.Reviews>': 'ta_reviews',
    'TA.Excellent': 'ta_excellent',
    'TA.Very Good': 'ta_very_good',
    'TA.Average': 'ta_average',
    'TA.Poor': 'ta_poor',
    'TA.Terrible': 'ta_terrible',
    'TA.Ratings>': 'ta_ratings',
    'TA.Location': 'ta_location',
    'TA.Cleanliness': 'ta_cleanliness',
    'TA.Service': 'ta_service',
    'TA.Value': 'ta_value',
    'GG.Reviews>': 'gg_reviews',
    'GG.Excellent': 'gg_excellent',
    'GG.Very Good': 'gg_very_good',
    'GG.Average': 'gg_average',
    'GG.Poor': 'gg_poor',
    'GG.Terrible': 'gg_terrible',
    'GG.Ratings>': 'gg_ratings',
    'GG.Location': 'gg_location',
    'GG.Cleanliness': 'gg_cleanliness',
    'GG.Service': 'gg_service',
    'GG.Value': 'gg_value',
    'EX.Reviews>': 'ex_reviews',
    'EX.Excellent': 'ex_excellent',
    'EX.Very Good': 'ex_very_good',
    'EX.Average': 'ex_average',
    'EX.Poor': 'ex_poor',
    'EX.Terrible': 'ex_terrible',
    'EX.Ratings>': 'ex_ratings',
    'EX.Location': 'ex_location',
    'EX.Cleanliness': 'ex_cleanliness',
    'EX.Service': 'ex_service',
    'EX.Value': 'ex_value',
    'BK.Reviews>': 'bk_reviews',
    'BK.Excellent': 'bk_excellent',
    'BK.Very Good': 'bk_very_good',
    'BK.Average': 'bk_average',
    'BK.Poor': 'bk_poor',
    'BK.Terrible': 'bk_terrible',
    'BK.Ratings>': 'bk_ratings',
    'BK.Location': 'bk_location',
    'BK.Cleanliness': 'bk_cleanliness',
    'BK.Service': 'bk_service',
    'BK.Value': 'bk_value',
    'Instagram': 'instagram',
    'Facebook': 'facebook',
    'Twitter': 'twitter'
}

df_clean = df_clean.rename(columns=column_mapping)

# Convertir week_date a formato DATE
df_clean['week_date'] = pd.to_datetime(df_clean['week_date'], format='%d/%m/%Y', errors='coerce')
df_clean['week_date'] = df_clean['week_date'].apply(lambda x: x.date() if pd.notna(x) else None)

# Convertir columnas numéricas que DEBEN ser enteros
integer_cols = ['week', 'series', 'rank_all', 'total',
                'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                'instagram', 'facebook', 'twitter']

for col in integer_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        df_clean[col] = df_clean[col].round(0).astype('Int64')

# Convertir columnas DECIMAL
decimal_cols = ['ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value']

for col in decimal_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print(f"Datos preparados: {len(df_clean)} filas")
print("\nMuestra de datos limpios:")
print(df_clean.head(2))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# Preparar datos
columns_order = ['week', 'week_date', 'active_selection', 'cs', 'series', 'cs_hotels_names',
                 'rank_all', 'total', 'market',
                 'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                 'ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                 'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                 'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                 'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                 'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                 'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                 'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value',
                 'instagram', 'facebook', 'twitter']

# Convertir DataFrame a lista de tuplas manejando pd.NA correctamente
data = []
for _, row in df_clean[columns_order].iterrows():
    tuple_data = []
    for val in row:
        if pd.isna(val) or val is pd.NA:
            tuple_data.append(None)
        else:
            tuple_data.append(val)
    data.append(tuple(tuple_data))

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.data_consolidado_historico
(week, week_date, active_selection, cs, series, cs_hotels_names,
 rank_all, total, market,
 ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible,
 ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value,
 gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible,
 gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value,
 ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible,
 ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value,
 bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible,
 bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value,
 instagram, facebook, twitter)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("\nInsertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n{len(data)} filas insertadas exitosamente")

cursor.close()
conn.close()

print("\nMigracion completada!")

Datos preparados: 20638 filas

Muestra de datos limpios:
     week   week_date active_selection                           Audit for:  \
0  220801  2022-08-01              Yes  Dreams Bahia Mita Surf & Spa Resort   
1  220801  2022-08-01               No  Dreams Bahia Mita Surf & Spa Resort   

       cs  series                             cs_hotels_names  rank_all  \
0  Direct       1   Dreams Bahia Mita Surf & Spa Resort (363)         5   
1  Direct       2  Secrets Bahia Mita Surf & Spa Resort (278)         6   

   total         market  ...  bk_poor  bk_terrible  bk_ratings  bk_location  \
0     15  Punta de Mita  ...        5            3         8.0          8.2   
1     15  Punta de Mita  ...        2            3         7.7          7.9   

   bk_cleanliness  bk_service  bk_value  instagram  facebook  twitter  
0             8.6         8.6       7.5       5038     10627       38  
1             8.9         8.0       7.4       4933      9766     <NA>  

[2 rows x 57 columns]

I

UniqueViolation: duplicate key value violates unique constraint "data_consolidado_historico_week_week_date_cs_hotels_names_key"
DETAIL:  Key (week, week_date, cs_hotels_names)=(211018, 2021-10-18, Dreams Bahia Mita Surf & Spa Resort (363)) already exists.


In [10]:
# Recargar CSV desde cero
df = pd.read_csv('data_consolidado.csv', encoding='utf-8', on_bad_lines='warn', engine='python')

# Aplicar todas las transformaciones SIN eliminar "duplicados"
# (todo el código de limpieza anterior EXCEPTO la parte de drop_duplicates)

In [11]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

# Recargar CSV completo desde cero
df = pd.read_csv('data_consolidado.csv',
                 encoding='utf-8',
                 on_bad_lines='warn',
                 engine='python')

df_clean = df.copy()

# Eliminar columna vacía
df_clean = df_clean.drop(columns=['Unnamed: 57'], errors='ignore')

# Renombrar columnas
column_mapping = {
    'Week': 'week', 'WeekDate': 'week_date', 'GFG': 'active_selection',
    'CS': 'cs', 'Series': 'series', 'CS Hotels Names': 'cs_hotels_names',
    'Rank All': 'rank_all', 'Total': 'total', 'Market': 'market',
    'TA.Reviews>': 'ta_reviews', 'TA.Excellent': 'ta_excellent', 'TA.Very Good': 'ta_very_good',
    'TA.Average': 'ta_average', 'TA.Poor': 'ta_poor', 'TA.Terrible': 'ta_terrible',
    'TA.Ratings>': 'ta_ratings', 'TA.Location': 'ta_location', 'TA.Cleanliness': 'ta_cleanliness',
    'TA.Service': 'ta_service', 'TA.Value': 'ta_value',
    'GG.Reviews>': 'gg_reviews', 'GG.Excellent': 'gg_excellent', 'GG.Very Good': 'gg_very_good',
    'GG.Average': 'gg_average', 'GG.Poor': 'gg_poor', 'GG.Terrible': 'gg_terrible',
    'GG.Ratings>': 'gg_ratings', 'GG.Location': 'gg_location', 'GG.Cleanliness': 'gg_cleanliness',
    'GG.Service': 'gg_service', 'GG.Value': 'gg_value',
    'EX.Reviews>': 'ex_reviews', 'EX.Excellent': 'ex_excellent', 'EX.Very Good': 'ex_very_good',
    'EX.Average': 'ex_average', 'EX.Poor': 'ex_poor', 'EX.Terrible': 'ex_terrible',
    'EX.Ratings>': 'ex_ratings', 'EX.Location': 'ex_location', 'EX.Cleanliness': 'ex_cleanliness',
    'EX.Service': 'ex_service', 'EX.Value': 'ex_value',
    'BK.Reviews>': 'bk_reviews', 'BK.Excellent': 'bk_excellent', 'BK.Very Good': 'bk_very_good',
    'BK.Average': 'bk_average', 'BK.Poor': 'bk_poor', 'BK.Terrible': 'bk_terrible',
    'BK.Ratings>': 'bk_ratings', 'BK.Location': 'bk_location', 'BK.Cleanliness': 'bk_cleanliness',
    'BK.Service': 'bk_service', 'BK.Value': 'bk_value',
    'Instagram': 'instagram', 'Facebook': 'facebook', 'Twitter': 'twitter'
}

df_clean = df_clean.rename(columns=column_mapping)

# Convertir fechas
df_clean['week_date'] = pd.to_datetime(df_clean['week_date'], format='%d/%m/%Y', errors='coerce')
df_clean['week_date'] = df_clean['week_date'].apply(lambda x: x.date() if pd.notna(x) else None)

# Convertir enteros
integer_cols = ['week', 'series', 'rank_all', 'total',
                'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                'instagram', 'facebook', 'twitter']

for col in integer_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').round(0).astype('Int64')

# Convertir decimales
decimal_cols = ['ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value']

for col in decimal_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print(f"Datos preparados: {len(df_clean)} filas")

# Conectar
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

columns_order = ['week', 'week_date', 'active_selection', 'cs', 'series', 'cs_hotels_names',
                 'rank_all', 'total', 'market',
                 'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                 'ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                 'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                 'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                 'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                 'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                 'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                 'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value',
                 'instagram', 'facebook', 'twitter']

data = []
for _, row in df_clean[columns_order].iterrows():
    tuple_data = []
    for val in row:
        if pd.isna(val) or val is pd.NA:
            tuple_data.append(None)
        else:
            tuple_data.append(val)
    data.append(tuple(tuple_data))

insert_sql = """
INSERT INTO raw_digital_presence.data_consolidado_historico
(week, week_date, active_selection, cs, series, cs_hotels_names,
 rank_all, total, market,
 ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible,
 ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value,
 gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible,
 gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value,
 ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible,
 ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value,
 bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible,
 bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value,
 instagram, facebook, twitter)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s)
"""

cursor = conn.cursor()
print("Insertando datos...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"{len(data)} filas insertadas exitosamente")

cursor.close()
conn.close()

print("Migracion completada!")

Datos preparados: 20638 filas
Insertando datos...


UniqueViolation: duplicate key value violates unique constraint "data_consolidado_historico_unique_key"
DETAIL:  Key (week, week_date, cs_hotels_names, series)=(211018, 2021-10-18, The St. Regis Punta Mita Resort (120), 3) already exists.


In [12]:
# Verificar duplicados con las 4 columnas del constraint
duplicados_reales = df_clean.duplicated(subset=['week', 'week_date', 'cs_hotels_names', 'series'], keep=False)

print(f"Filas duplicadas (considerando series): {duplicados_reales.sum()}")

if duplicados_reales.sum() > 0:
    print("\nPrimeras 20 filas duplicadas:")
    print(df_clean[duplicados_reales][['week', 'week_date', 'cs_hotels_names', 'series', 'rank_all']].head(20))

    # Ver si son COMPLETAMENTE idénticas
    duplicados_completos = df_clean.duplicated(keep=False)
    print(f"\nFilas COMPLETAMENTE idénticas: {duplicados_completos.sum()}")

    # Eliminar duplicados manteniendo la primera ocurrencia
    print("\nEliminando duplicados...")
    df_clean = df_clean.drop_duplicates(subset=['week', 'week_date', 'cs_hotels_names', 'series'], keep='first')
    print(f"Filas después de limpiar: {len(df_clean)}")

Filas duplicadas (considerando series): 6969

Primeras 20 filas duplicadas:
      week   week_date                               cs_hotels_names  series  \
1   220801  2022-08-01    Secrets Bahia Mita Surf & Spa Resort (278)       2   
2   220801  2022-08-01         The St. Regis Punta Mita Resort (120)       3   
3   220801  2022-08-01          Four Seasons Resort Punta Mita (205)       4   
4   220801  2022-08-01                         W Punta de Mita (119)       5   
5   220801  2022-08-01                    Conrad Punta de Mita (324)       6   
6   220801  2022-08-01                                  Casa De Mita       7   
7   220801  2022-08-01                       Hotel La Quinta del Sol       8   
8   220801  2022-08-01                Four Seasons Resort Punta Mita       9   
9   220801  2022-08-01         Marival Armony Luxury Resort & Suites      10   
10  220801  2022-08-01  Dreams Bahia Mita Surf & Spa Resort ( 363 )       11   
12  220725  2022-07-25    Secrets Bahia Mita

In [13]:
# Verificación exhaustiva de duplicados
print("=== ANÁLISIS DE DUPLICADOS ===\n")

# 1. Ver filas duplicadas completas
duplicados_4col = df_clean.duplicated(subset=['week', 'week_date', 'cs_hotels_names', 'series'], keep=False)
df_duplicados = df_clean[duplicados_4col].sort_values(['week', 'week_date', 'cs_hotels_names', 'series'])

print(f"Total filas marcadas como duplicadas: {duplicados_4col.sum()}")

# 2. Tomar un caso específico para analizar
caso_ejemplo = df_duplicados.head(2)
print("\n=== EJEMPLO: Primeras 2 filas duplicadas (TODAS las columnas) ===")
print(caso_ejemplo.to_string())

# 3. Ver si son IDÉNTICAS en todas las columnas
print("\n=== ¿Son COMPLETAMENTE idénticas? ===")
duplicados_completos = df_clean.duplicated(keep=False)
print(f"Filas 100% idénticas en TODAS las columnas: {duplicados_completos.sum()}")

# 4. Si NO son idénticas, ver qué columnas difieren
if duplicados_completos.sum() == 0 and duplicados_4col.sum() > 0:
    print("\n⚠️ ALERTA: Las filas NO son idénticas - difieren en otras columnas")
    print("\nMostrando diferencias en las primeras 10 filas duplicadas:")

    # Agrupar por las 4 columnas clave y contar cuántas filas hay por grupo
    grupos = df_clean[duplicados_4col].groupby(['week', 'week_date', 'cs_hotels_names', 'series']).size()
    print(f"\nGrupos de duplicados: {len(grupos)}")
    print(f"Filas por grupo (primeros 5):")
    print(grupos.head())

=== ANÁLISIS DE DUPLICADOS ===

Total filas marcadas como duplicadas: 0

=== EJEMPLO: Primeras 2 filas duplicadas (TODAS las columnas) ===
Empty DataFrame
Columns: [week, week_date, active_selection, Audit for:, cs, series, cs_hotels_names, rank_all, total, market, ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible, ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value, gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible, gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value, ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible, ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value, bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible, bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value, instagram, facebook, twitter]
Index: []

=== ¿Son COMPLETAMENTE idénticas? ===
Filas 100% idénticas en TODAS las columnas: 0


In [14]:
# Recargar CSV ORIGINAL sin tocar
df_original = pd.read_csv('data_consolidado.csv',
                          encoding='utf-8',
                          on_bad_lines='warn',
                          engine='python')

# Aplicar solo renombrado de columnas (sin eliminar nada)
column_mapping = {
    'Week': 'week', 'WeekDate': 'week_date', 'GFG': 'active_selection',
    'CS': 'cs', 'Series': 'series', 'CS Hotels Names': 'cs_hotels_names',
    'Rank All': 'rank_all', 'Total': 'total', 'Market': 'market'
}

df_check = df_original.rename(columns=column_mapping)

# Convertir fechas para comparación correcta
df_check['week_date'] = pd.to_datetime(df_check['week_date'], format='%d/%m/%Y', errors='coerce')

# Buscar duplicados basados en las 4 columnas clave
duplicados = df_check.duplicated(subset=['week', 'week_date', 'cs_hotels_names', 'series'], keep=False)

print(f"Total filas duplicadas: {duplicados.sum()}")

if duplicados.sum() > 0:
    # Ver las primeras 10 filas duplicadas CON TODAS LAS COLUMNAS
    df_dup = df_check[duplicados].sort_values(['week', 'week_date', 'cs_hotels_names', 'series'])

    print("\n=== Primeras 10 filas duplicadas (TODAS las columnas) ===")
    print(df_dup[['week', 'week_date', 'cs_hotels_names', 'series', 'rank_all',
                  'TA.Reviews>', 'TA.Ratings>', 'Instagram', 'Facebook']].head(10))

    # Verificar si son COMPLETAMENTE idénticas
    duplicados_completos = df_check.duplicated(keep=False)
    print(f"\n¿Son 100% idénticas en TODAS las columnas?: {duplicados_completos.sum()} filas")

    if duplicados_completos.sum() == duplicados.sum():
        print("✅ SÍ - Son duplicados reales (todas las columnas idénticas)")
    else:
        print("⚠️ NO - Solo comparten las 4 columnas clave, pero difieren en otras")

Total filas duplicadas: 6969

=== Primeras 10 filas duplicadas (TODAS las columnas) ===
        week  week_date                 cs_hotels_names  series  rank_all  \
435   211018 2021-10-18                    Casa De Mita     7.0       1.0   
446   211018 2021-10-18                    Casa De Mita     7.0       1.0   
6925  211018 2021-10-18                    Casa De Mita     7.0       1.0   
438   211018 2021-10-18            Conrad Punta de Mita    10.0       4.0   
449   211018 2021-10-18            Conrad Punta de Mita    10.0       4.0   
6928  211018 2021-10-18            Conrad Punta de Mita    10.0       4.0   
434   211018 2021-10-18      Conrad Punta de Mita (324)     6.0       4.0   
445   211018 2021-10-18      Conrad Punta de Mita (324)     6.0       4.0   
6924  211018 2021-10-18      Conrad Punta de Mita (324)     6.0       4.0   
439   211018 2021-10-18  Four Seasons Resort Punta Mita    11.0       5.0   

     TA.Reviews>  TA.Ratings> Instagram Facebook  
435          

In [15]:
import psycopg2
from psycopg2.extras import execute_batch

# df_clean ya tiene las 16,708 filas sin duplicados

# Preparar datos
columns_order = ['week', 'week_date', 'active_selection', 'cs', 'series', 'cs_hotels_names',
                 'rank_all', 'total', 'market',
                 'ta_reviews', 'ta_excellent', 'ta_very_good', 'ta_average', 'ta_poor', 'ta_terrible',
                 'ta_ratings', 'ta_location', 'ta_cleanliness', 'ta_service', 'ta_value',
                 'gg_reviews', 'gg_excellent', 'gg_very_good', 'gg_average', 'gg_poor', 'gg_terrible',
                 'gg_ratings', 'gg_location', 'gg_cleanliness', 'gg_service', 'gg_value',
                 'ex_reviews', 'ex_excellent', 'ex_very_good', 'ex_average', 'ex_poor', 'ex_terrible',
                 'ex_ratings', 'ex_location', 'ex_cleanliness', 'ex_service', 'ex_value',
                 'bk_reviews', 'bk_excellent', 'bk_very_good', 'bk_average', 'bk_poor', 'bk_terrible',
                 'bk_ratings', 'bk_location', 'bk_cleanliness', 'bk_service', 'bk_value',
                 'instagram', 'facebook', 'twitter']

# Convertir a tuplas manejando pd.NA
data = []
for _, row in df_clean[columns_order].iterrows():
    tuple_data = []
    for val in row:
        if pd.isna(val) or val is pd.NA:
            tuple_data.append(None)
        else:
            tuple_data.append(val)
    data.append(tuple(tuple_data))

print(f"Preparando inserción de {len(data)} filas...")

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.data_consolidado_historico
(week, week_date, active_selection, cs, series, cs_hotels_names,
 rank_all, total, market,
 ta_reviews, ta_excellent, ta_very_good, ta_average, ta_poor, ta_terrible,
 ta_ratings, ta_location, ta_cleanliness, ta_service, ta_value,
 gg_reviews, gg_excellent, gg_very_good, gg_average, gg_poor, gg_terrible,
 gg_ratings, gg_location, gg_cleanliness, gg_service, gg_value,
 ex_reviews, ex_excellent, ex_very_good, ex_average, ex_poor, ex_terrible,
 ex_ratings, ex_location, ex_cleanliness, ex_service, ex_value,
 bk_reviews, bk_excellent, bk_very_good, bk_average, bk_poor, bk_terrible,
 bk_ratings, bk_location, bk_cleanliness, bk_service, bk_value,
 instagram, facebook, twitter)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("Insertando datos en PostgreSQL...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n✅ {len(data)} filas insertadas exitosamente en data_consolidado_historico")

# Cerrar conexión
cursor.close()
conn.close()

print("\n🎉 Migración completada!")

Preparando inserción de 16708 filas...
Insertando datos en PostgreSQL...

✅ 16708 filas insertadas exitosamente en data_consolidado_historico

🎉 Migración completada!
